<a href="https://colab.research.google.com/github/19Alireza88/Summarization/blob/main/multi_model_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Project_1/new_data.csv', encoding='windows-1252')

# **Model: Qwen/Qwen2.5-0.5B-Instruct**

In [5]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [8]:
summ = list()
instruction = "You are Qwen, an advanced language model specializing in summarization."

for title, abstract in zip(data['Title'], data['Abstract']):
    prompt = f'''Using the provided title and abstract of the article, generate a one-sentence summary that captures the core idea and purpose of the research.
Title: {title}
Abstract: {abstract}
One sentence summary: '''
    messages = [
        {"role": "system", "content": "You are Qwen, an advanced language model specializing in summarization."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    # Tokenize the input
    model_inputs = tokenizer([text], return_tensors="pt")
    model_inputs = {key: tensor.to(model.device) for key, tensor in model_inputs.items()}

    # Generate the output
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    # Exclude the input IDs from the generated output
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs["input_ids"], generated_ids)
    ]

    # Decode and append the summaries
    summ.append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])


In [9]:
data['Qwen/Qwen2.5-0.5B-Instruct'] = summ

# **Model: Qwen/Qwen2.5-3B-Instruct**

In [11]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [13]:
summ = list()
instruction = "You are Qwen, an advanced language model specializing in summarization."

for title, abstract in zip(data['Title'], data['Abstract']):
  prompt = f'''Using the provided title and abstract of the article, generate a one-sentence summary that captures the core idea and purpose of the research.
Title: {title}
Abstract: {abstract}
One sentence summary: '''
  messages = [
    {"role": "system", "content": "You are Qwen, an advanced language model specializing in summarization."},
    {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt")
  model_inputs = {key: tensor.to(model.device) for key, tensor in model_inputs.items()}

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs['input_ids'], generated_ids)
  ]

  summ.append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

In [14]:
data['Qwen/Qwen2.5-3B-Instruct'] = summ

# **Model: microsoft/Phi-3.5-mini-instruct**

In [16]:
torch.random.manual_seed(0)

model_name = "microsoft/Phi-3.5-mini-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [17]:
summ = list()
instruction = "You are a helpful AI assistant specialized in academic writing and summarizing."

for title, abstract in zip(data['Title'], data['Abstract']):
  prompt = f'''Using the provided title and abstract of the article, generate a one-sentence summary that captures the core idea and purpose of the research.
Title: {title}
Abstract: {abstract}
One sentence summary: '''
  messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": prompt}
  ]
  pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
  )

  generation_args = {
      "max_new_tokens": 512,
      "return_full_text": False,
      "temperature": 0.2,
      "do_sample": False,
  }

  output = pipe(messages, **generation_args)
  summ.append(output[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


In [18]:
data['microsoft/Phi-3.5-mini-instruct'] = summ

In [19]:
data.to_csv('new_data_summ_new_prompt.csv', index=False)